<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/PDSH-cover-small.png?raw=1">

*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Vectorized String Operations](03.10-Working-With-Strings.ipynb) | [Contents](Index.ipynb) | [High-Performance Pandas: eval() and query()](03.12-Performance-Eval-and-Query.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.11-Working-with-Time-Series.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


# 使用時間序列資料

在財務模型領域中Pandas也發展了許多，也包含了很多工具集可以用來操作dates和times資料。

- *Time stamps*表示時間中某一個特定的時刻(例如:2015年7月4日早上0700)
- *Time intervals*表示某一個特定的開始時間和結束時間中的時間長度(例如:2015年)
- *periods*表示一個時間間隔的特例，每一個間隔是一個固定的長度，且不會重疊(例如:使用24小時的長度來構成一天)
- *Time deltas*或*durations*表示一個時間長度(例如:22.56秒長).

本節會介紹如何在Python或Pandas使用這裡的每一個date/time資料。

## 在Python中的Date和Time

### Python原生的date和time:``datetime``和``dateutil``

Python用來操作date和time的基本物件是內建的date和time模組，再搭配第三方 ``datetime``套件，可以很快執行一堆在date和time上有用的功能。


In [ ]:
# 用datetime型態動手建立日期
from datetime import datetime
datetime(year=2015, month=7, day=4)

datetime.datetime(2015, 7, 4, 0, 0)

In [ ]:
# 用dateutil從各式各樣的字串剖析出日期
from dateutil import parser
date = parser.parse("4th of July, 2015")
date

datetime.datetime(2015, 7, 4, 0, 0)

``datetime``和``dateutil``的威力在於它們的使用彈性和簡易的語法:可以使用這些物件和它們內建的方法，很容易的去執行幾乎任何感興趣的運算。而當打算操作非常大的date和time陣列時，就像是Python的數值變數list會比Numpy型態的數值陣列效能差一樣，Python之datetime物件的list也會比編碼過的日期固定型態陣列來的差。

### 時間的定型陣列:NumPy的``datetime64``

``datetime64`` dtype把日期編碼成64位元的整數，如此讓日期的陣列可以被表示的非常緊密。datetime64需要一個非常獨特的輸入格式。

In [ ]:
# 轉換成datetime64格式化的日期
import numpy as np
date = np.array('2015-07-04', dtype=np.datetime64)
date

array('2015-07-04', dtype='datetime64[D]')

In [ ]:
# 格式化後，可進行向量化的操作
date + np.arange(12)

array(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
       '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
       '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
      dtype='datetime64[D]')

因為NumPy``datetime64``陣列是單一型態，這個型式的操作可以比在Python中使用``datetime``物件還要更快的被執行，特別是陣列變得更大的時候。

``datetime64``和``timedelta64``物件有一個細節是，它們被建立在一個*基本的時間單位*上。因為``datetime64``物件被限制在64位元的精確度，此編碼的範圍為$2^{64}$乘上這個基本單位。換句話說，``datetime64``強制在*時間的解析度*和*最大可表示範圍*間做了取捨。

例如:如果想要可以達到一奈秒的精確度，你只能編碼到$2^{64}$個奈秒，也就是不到600年。NumPy會從輸入值去推算想要使用的單位，如下:

In [ ]:
# 以天為單位的datetime
np.datetime64('2015-07-04')

numpy.datetime64('2015-07-04')

In [ ]:
# 以分為單位的datetime
np.datetime64('2015-07-04 12:00')

numpy.datetime64('2015-07-04T12:00')

The following table, drawn from the [NumPy datetime64 documentation](http://docs.scipy.org/doc/numpy/reference/arrays.datetime.html), 列出可用的格式碼以及該編碼可以使用的相對和絕對時間範圍:

|編碼    | 意義     | 時間範圍 (相對) | 時間範例 (絕對)   |
|--------|-------------|----------------------|------------------------|
| ``Y``  | Year	       | ± 9.2e18 years       | [9.2e18 BC, 9.2e18 AD] |
| ``M``  | Month       | ± 7.6e17 years       | [7.6e17 BC, 7.6e17 AD] |
| ``W``  | Week	       | ± 1.7e17 years       | [1.7e17 BC, 1.7e17 AD] |
| ``D``  | Day         | ± 2.5e16 years       | [2.5e16 BC, 2.5e16 AD] |
| ``h``  | Hour        | ± 1.0e15 years       | [1.0e15 BC, 1.0e15 AD] |
| ``m``  | Minute      | ± 1.7e13 years       | [1.7e13 BC, 1.7e13 AD] |
| ``s``  | Second      | ± 2.9e12 years       | [ 2.9e9 BC, 2.9e9 AD]  |
| ``ms`` | Millisecond | ± 2.9e9 years        | [ 2.9e6 BC, 2.9e6 AD]  |
| ``us`` | Microsecond | ± 2.9e6 years        | [290301 BC, 294241 AD] |
| ``ns`` | Nanosecond  | ± 292 years          | [ 1678 AD, 2262 AD]    |
| ``ps`` | Picosecond  | ± 106 days           | [ 1969 AD, 1970 AD]    |
| ``fs`` | Femtosecond | ± 2.6 hours          | [ 1969 AD, 1970 AD]    |
| ``as`` | Attosecond  | ± 9.2 seconds        | [ 1969 AD, 1970 AD]    |

雖然``datetime64``資料型態解決了一些Python內建的``datetime``型態的不足，但它缺少了許多方便的函式和方法，如``datetime``和特別是``dateutil``所提供的。
更多資訊可在[NumPy's datetime64 documentation](http://docs.scipy.org/doc/numpy/reference/arrays.datetime.html).

### 在Pandas中的date和time: 在兩個世界都最好

Pandas建立了之前所有提到過的工具以提供``Timestamp``物件，它結合了``datetime``和``dateutil``的易用性以及``numpy.datetime64``的儲存效率及向量化的介面。從這些``Timestamp``物件，Pandas可以建立一個``DatetimeIndex``，它可以被使用在``Series``或``DataFrame``的索引資料。

In [ ]:
# 剖析一個具有彈性的格式字串資料，以及使用格式碼輸出某一天是星期幾:
import pandas as pd
date = pd.to_datetime("4th of July, 2015")
date

Timestamp('2015-07-04 00:00:00')

可以直接在同一個物件上執行Numpy型態的向量化操作:

In [ ]:
date + pd.to_timedelta(np.arange(12), 'D')

DatetimeIndex(['2015-07-04', '2015-07-05', '2015-07-06', '2015-07-07',
               '2015-07-08', '2015-07-09', '2015-07-10', '2015-07-11',
               '2015-07-12', '2015-07-13', '2015-07-14', '2015-07-15'],
              dtype='datetime64[ns]', freq=None)

## Pandas時間序列: 使用時間做索引

*使用時間標記來索引資料*時，Pandas的時間序列工具會變得很有用。

In [ ]:
# 建構拒時間索引資料的Series物件
index = pd.DatetimeIndex(['2014-07-04', '2014-08-04',
                          '2015-07-04', '2015-08-04'])
data = pd.Series([0, 1, 2, 3], index=index)
data

2014-07-04    0
2014-08-04    1
2015-07-04    2
2015-08-04    3
dtype: int64

In [ ]:
# 可用Series索引樣式，傳遞被包裝成日期的值
data['2014-07-04':'2015-07-04']

2014-07-04    0
2014-08-04    1
2015-07-04    2
dtype: int64

In [ ]:
# 傳遞某年可取得所有該年度的資料
data['2015']

2015-07-04    2
2015-08-04    3
dtype: int64

## Pandas時間序列資料結構

本節會介紹和時間序列資料一起使用的基本資料結構:
- 對*time stamps*: Pandas提供``Timestamp``型態。取代Python原生的``datetime``, 但建立在更有效率的``numpy.datetime64``資料型態上。其相關的索引結構是``DatetimeIndex``
- 對*time Periods*: Pandas提供``Period``型態。以``numpy.datetime64``為基礎編碼了一個固定頻率間隔。其相關的索引結構是``PeriodIndex``
- 對*time deltas*或*durations*: Pandas提供``Timedelta``型態。``Timedelta``是Python原生``datetime.timedelta``型態更有效率的替代品，也是以``numpy.timedelta64``為基礎。其相關的索引結構是``TimedeltaIndex``

在這些日期時間物件中，最基本的是``Timestamp``和``DatetimeIndex``物件。雖然這些類別物件可以直接被呼叫，但是最常使用的是``pd.to_datetime()``函數，它可以解析非常多種格式。



In [ ]:
# 傳遞一個單一日期到``pd.to_datetime()``會產生一個``Timestamp``，傳遞一串日期時預設會產生一個``DatetimeIndex``
dates = pd.to_datetime([datetime(2015, 7, 3), '4th of July, 2015',
                       '2015-Jul-6', '07-07-2015', '20150708'])
dates

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-06', '2015-07-07',
               '2015-07-08'],
              dtype='datetime64[ns]', freq=None)

In [ ]:
# 當一個日期減去另一個時，會產生一個``TimedeltaIndex``:
dates - dates[0]

TimedeltaIndex(['0 days', '1 days', '3 days', '4 days', '5 days'], dtype='timedelta64[ns]', freq=None)

### 規則性的序列: ``pd.date_range()``

為了讓建立規則的日期序列更方便，Pandas提供一些函式來做這件事:``pd.date_range()``可產生timestamps, ``pd.period_range()``可產生periods, and ``pd.timedelta_range()``可產生time deltas。

``pd.date_range()``也可接受起始日期、結束日期以及可以選用的頻率編碼，然後創建一個規則的日期序列。

In [ ]:
# 預設是以1天為單位
pd.date_range('2015-07-03', '2015-07-10')

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-05', '2015-07-06',
               '2015-07-07', '2015-07-08', '2015-07-09', '2015-07-10'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
# 起始日+重複次數
pd.date_range('2015-07-03', periods=8)

DatetimeIndex(['2015-07-03', '2015-07-04', '2015-07-05', '2015-07-06',
               '2015-07-07', '2015-07-08', '2015-07-09', '2015-07-10'],
              dtype='datetime64[ns]', freq='D')

In [ ]:
# 更改頻率
pd.date_range('2015-07-03', periods=8, freq='H')

DatetimeIndex(['2015-07-03 00:00:00', '2015-07-03 01:00:00',
               '2015-07-03 02:00:00', '2015-07-03 03:00:00',
               '2015-07-03 04:00:00', '2015-07-03 05:00:00',
               '2015-07-03 06:00:00', '2015-07-03 07:00:00'],
              dtype='datetime64[ns]', freq='H')

In [ ]:
# 建立Periods的規則
pd.period_range('2015-07', periods=8, freq='M')

PeriodIndex(['2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
             '2016-01', '2016-02'],
            dtype='period[M]')

In [ ]:
# 建立delta時間序列的規則
pd.timedelta_range(0, periods=10, freq='H')

TimedeltaIndex(['0 days 00:00:00', '0 days 01:00:00', '0 days 02:00:00',
                '0 days 03:00:00', '0 days 04:00:00', '0 days 05:00:00',
                '0 days 06:00:00', '0 days 07:00:00', '0 days 08:00:00',
                '0 days 09:00:00'],
               dtype='timedelta64[ns]', freq='H')

## 頻率和位移值(Offsets)

Pandas時間系列工具的基礎是關於頻率和日期偏移值的概念，如``D``(day)和``H`` (hour)編碼，可以使用這些編碼去指定任何想要的頻率間隔。 

| 編碼   | 明天         | 編碼   | 說明          |
|--------|---------------------|--------|----------------------|
| ``D``  | Calendar day        | ``B``  | Business day         |
| ``W``  | Weekly              |        |                      |
| ``M``  | Month end           | ``BM`` | Business month end   |
| ``Q``  | Quarter end         | ``BQ`` | Business quarter end |
| ``A``  | Year end            | ``BA`` | Business year end    |
| ``H``  | Hours               | ``BH`` | Business hours       |
| ``T``  | Minutes             |        |                      |
| ``S``  | Seconds             |        |                      |
| ``L``  | Milliseonds         |        |                      |
| ``U``  | Microseconds        |        |                      |
| ``N``  | nanoseconds         |        |                      |

其中月、季、年頻率都是用來標記指定的period的結束點，加上一個``S``的字尾到其中任一個表示它是開始點。

| 編碼    | 說明            || 編碼    | 說明            |
|---------|------------------------||---------|------------------------|
| ``MS``  | Month start            ||``BMS``  | Business month start   |
| ``QS``  | Quarter start          ||``BQS``  | Business quarter start |
| ``AS``  | Year start             ||``BAS``  | Business year start    |

此外可以加上3個字母的月編碼放在字尾，標記任何季或年編碼所使用的月份:
- ``Q-JAN``, ``BQ-FEB``, ``QS-MAR``, ``BQS-APR``等等
- ``A-JAN``, ``BA-FEB``, ``AS-MAR``, ``BAS-APR``等等

同樣的方式，也可以加上3個字母的週編碼去作為週頻率的分割點:
- ``W-SUN``, ``W-MON``, ``W-TUE``, ``W-WED``等等

程式也可以結合數字以指定其他的頻率。

In [ ]:
# 結合小時(H)以及分(T)編碼
pd.timedelta_range(0, periods=9, freq="2H30T")

TimedeltaIndex(['0 days 00:00:00', '0 days 02:30:00', '0 days 05:00:00',
                '0 days 07:30:00', '0 days 10:00:00', '0 days 12:30:00',
                '0 days 15:00:00', '0 days 17:30:00', '0 days 20:00:00'],
               dtype='timedelta64[ns]', freq='150T')

所有這些程式可以被用來作為Pandas時間序列位移值的特定實例，可在``pd.tseries.offsets``模組中找到。


In [ ]:
# 建立一個工作日偏移值
from pandas.tseries.offsets import BDay
pd.date_range('2015-07-01', periods=5, freq=BDay())

DatetimeIndex(['2015-07-01', '2015-07-02', '2015-07-03', '2015-07-06',
               '2015-07-07'],
              dtype='datetime64[ns]', freq='B')